In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets


In [3]:
#devic agnostic code preperation
device = 'cpu'
if torch.cuda.is_available():
  device = 'cuda'

In [4]:
train_images = torchvision.datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = torchvision.transforms.ToTensor(),
    target_transform = None
)
#train_images.to("cuda")
test_images = torchvision.datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = torchvision.transforms.ToTensor(),
    target_transform = None
)
#test_images.to("cuda")

100%|██████████| 26421880/26421880 [00:01<00:00, 17515229.53it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 333141.05it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6081045.94it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4427368.67it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
image, lable = train_images[0]
image.shape

torch.Size([1, 28, 28])

In [5]:
dataloader = torch.utils.data.DataLoader(dataset = train_images,
                             batch_size = 32,
                             shuffle = True,
                             )
test_dataloader = torch.utils.data.DataLoader(dataset = test_images,
                                              batch_size = 32,
                                              shuffle = True
                                              )

In [ ]:
count = 0
for i,j in test_dataloader:
  print(i)
  print("##########")
  print(j)
  if count == 4:
    break
  count += 1

In [6]:
class Image_Classification_Model(nn.Module):
    def __init__(self, input_size, output_size):
      super().__init__()
      self.layer_stack = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features = input_size, out_features = input_size * 2),
          nn.ReLU(),
          nn.Linear(in_features = input_size * 2, out_features = output_size),
          nn.ReLU()
      )

    def forward(self, input):
        return self.layer_stack(input)



In [15]:
#creating the model
model = Image_Classification_Model(28**2, 10)
model.to(device)

Image_Classification_Model(
  (layer_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=1568, bias=True)
    (2): ReLU()
    (3): Linear(in_features=1568, out_features=10, bias=True)
    (4): ReLU()
  )
)

In [16]:
#Setting up loss fucntion and the optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

In [ ]:
model.state_dict()

In [54]:
def accuracy(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [17]:
from tqdm.auto import tqdm


In [51]:
#Making the training loop

epochs = 5
for i in range(epochs):
  total_loss = 0
  model.train()
  for batch, (x, y) in enumerate(dataloader):
    model.train()
    x = x.to(device)
    y = y.to(device)

    y_pred = model(x)

    loss = loss_fn(y_pred, y)

    total_loss += loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % (len(dataloader) - 1) == 0 and batch != 0:
      print("batch number %d in epoch %d......" %(batch, i))
      #print("train loss --------- %f" %(total_loss/len(dataloader)))
      print("train loss --------- %f" %(loss))


    #testing section
      model.eval()
      with torch.inference_mode():
        total_test_loss = 0
        for x_test, y_test in test_dataloader:
          x_test = x_test.to(device)
          y_test = y_test.to(device)
          test_y_pred = model(x_test)
          test_y_probs = torch.softmax(test_y_pred, dim = 1)
          y_prediction = torch.argmax(test_y_probs, dim = 1)
          loss = loss_fn(test_y_pred, y_test)
          total_test_loss += loss
        print("test loss: --------- %f" %(total_test_loss/len(dataloader)))
        print("test accuracy: ------ %f" %(accuracy(y_test, y_prediction)))





batch number 1874 in epoch 0......
train loss --------- 0.628248
test loss: --------- 0.128117
test accuracy: ------ 75.000000
batch number 1874 in epoch 1......
train loss --------- 0.201165
test loss: --------- 0.073096
test accuracy: ------ 56.250000
batch number 1874 in epoch 2......
train loss --------- 0.173463
test loss: --------- 0.064385
test accuracy: ------ 100.000000
batch number 1874 in epoch 3......
train loss --------- 0.312550
test loss: --------- 0.060793
test accuracy: ------ 93.750000
batch number 1874 in epoch 4......
train loss --------- 0.649001
test loss: --------- 0.062259
test accuracy: ------ 100.000000


In [19]:
def train_it(model, dataloader, loss_fn, optimizer, print_interval = 20, epochs = 100):
    #devic agnostic code preperation
    device = 'cpu'
    if torch.cuda.is_available():
        device = 'cuda'

    def accuracy(y_true, y_pred):
        """Calculates accuracy between truth labels and predictions.

        Args:
            y_true (torch.Tensor): Truth labels for predictions.
            y_pred (torch.Tensor): Predictions to be compared to predictions.

        Returns:
            [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
        """
        correct = torch.eq(y_true, y_pred).sum().item()
        acc = (correct / len(y_pred)) * 100
        return acc

    #training loop
    for epoch in range(epochs):
        print("going through epoch: %d:" % (epoch))

        for batch_idx, (x_train, y_train) in enumerate(dataloader):
            model.train()

            #setting up hardware
            model.to(device)
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            # print(next(iter(k).device)
            # print(x_train.device)
            # print(y_train.device)
            #####

            y_pred = model(x_train)

            loss = loss_fn(y_pred, y_train)
            #total_loss += loss_fn

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_idx % print_interval == 0 and batch_idx != 0:
                print("  batch number ---------- %d...." %(batch_idx))
                print("  batch train loss ------ %f...." %(loss))
                y_probs = torch.softmax(y_pred, dim = 1)
                predictions = torch.argmax(y_probs, dim = 1)
                # print(predictions)
                print("  train accuracy -------- %f...." %(accuracy(y_train,
                                                                    predictions
                                                                    )))



# def test_it(model, dataloader):






In [20]:
train_it(model, dataloader, loss_fn, optimizer, print_interval = 1874, epochs = 5)

going through epoch: 0:
  batch number ---------- 1874....
  batch train loss ------ 0.841106....
  train accuracy -------- 71.875000....
going through epoch: 1:
  batch number ---------- 1874....
  batch train loss ------ 0.499399....
  train accuracy -------- 84.375000....
going through epoch: 2:
  batch number ---------- 1874....
  batch train loss ------ 0.568265....
  train accuracy -------- 78.125000....
going through epoch: 3:
  batch number ---------- 1874....
  batch train loss ------ 0.433304....
  train accuracy -------- 84.375000....
going through epoch: 4:
  batch number ---------- 1874....
  batch train loss ------ 0.420388....
  train accuracy -------- 81.250000....


In [18]:
logits = model(testing_x)
prob = torch.softmax(logits, dim = 1)
preds = torch.argmax(prob, dim = 1)
print(preds)

tensor([3, 7, 1, 1, 9, 3, 3, 9, 1, 3, 1, 1, 3, 3, 4, 1, 1, 7, 1, 3, 1, 3, 1, 3,
        3, 1, 7, 4, 3, 3, 7, 7])


In [13]:
testing_y

tensor([9, 4, 6, 4, 5, 4, 7, 8, 6, 2, 0, 4, 0, 4, 9, 3, 2, 5, 2, 2, 2, 6, 0, 4,
        3, 1, 5, 0, 0, 3, 9, 9])

In [21]:
new_logits = model(testing_x)
new_prob = torch.softmax(new_logits, dim = 1)
new_preds = torch.argmax(new_prob, dim = 1)
print(new_preds)

tensor([9, 4, 6, 4, 5, 4, 7, 8, 0, 4, 0, 4, 0, 4, 9, 3, 2, 5, 2, 6, 2, 6, 0, 4,
        3, 1, 5, 0, 6, 3, 9, 9])


In [10]:
testing_x = 0
testing_y = 0
for batch, (x,y) in enumerate(test_dataloader):
    testing_x = x
    testing_y = y
    break


In [ ]:
from tqdm.auto import tqdm

class PytorchModel(nn.Module):
    def __init__(self):
        super().__init__()

    def train_it(self, dataloader, loss_fn, epochs = 100):
        total_loss = 0
        for epoch in tqdm(range(epochs)):
            for batch_idx, (x_train, y_train) in enumerate(dataloader):
                #shifting to training mode
                self.train()

                # going through the forward method
                y_logits = self.forward(x_train)
                y_probs = torch.softmax(y_logits, dim = 1)
                y_result = torch.argmax(y_probs, dim = 1)


In [47]:
type(train_images)

torchvision.datasets.mnist.FashionMNIST